code: https://github.com/ritts97/Unsupervised-Face-Clustering

## Setup

In [2]:
import sys
import os
import dlib
import glob
import time

import cv2
import os

import matplotlib.pyplot as plt

from imagecluster import FaceClassifier

import random
import torch
import numpy as np

from imagecluster import calc, icio, postproc
# import icio
# import postproc

# from person_db import Person
# from person_db import Face
# from person_db import PersonDB

print(dlib.DLIB_USE_CUDA)
print(cv2.__version__)

True
4.5.5


In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore

seed_everything(79)

In [4]:
dlib.DLIB_USE_CUDA

True

In [5]:
import face_recognition

def check_use_gpu(TARGET_IMG_PATH):
    image = face_recognition.load_image_file(TARGET_IMG_PATH)
    face_locations = face_recognition.face_locations(image,model='cnn')
    if len(face_locations) > 0:
        print('Using GPU')
    else:
        print('***Not using GPU***')

TARGET_IMG_PATH = "/opt/ml/input/final-project-level3-cv-10/data/img1.png"
check_use_gpu(TARGET_IMG_PATH)

Using GPU


## Extract frame from video and save as images

In [6]:
video_paths = {
    0: "/opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s.mp4",
}

In [7]:
file_num = 0

filepath = video_paths[file_num]
video = cv2.VideoCapture(filepath)

assert video.isOpened(), f"Could not Open : {filepath}"

In [8]:
# 불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 7200
width : 1280
height : 720
fps : 24.0


In [9]:
# 변수 설정
frame_batch_size = 16
capture_interval = 3 * int(round(fps)) # n초 간격 프레임 캡쳐
stop = 300 # 0 초과의 정수 지정 시 n 초 까지만 영상 실행
skip = 0 # 초반 n 초 skip
face_cnt = 150 # 몇 장의 인물 사진을 사용할 지
capture_cnt = 60
threshold = 0.36 # 같은 인물로 판단할 threshold -> 낮을수록 같은 인물로 판단하지 않음
ratio = 1.0 # resizing ratio인데, 건드리지 않는게 좋을 듯

In [10]:
result_dir = filepath[:-4] + '_test2'
# pdb = PersonDB()
# pdb.load_db(result_dir)
# pdb.print_persons()

In [11]:
try:
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
except OSError:
    print('Error: Creaing directory. ' + result_dir)

**dnn_superres 사용**
```
pip uninstall opencv-python
pip uninstall opencv-contrib-python
pip3 install opencv-contrib-python
```
- 코드 및 성능비교표: https://learnopencv.com/super-resolution-in-opencv/ 
- super resolution 모델들 & 다운로드 링크: https://github.com/opencv/opencv_contrib/blob/master/modules/dnn_superres/README.md

In [12]:
video = cv2.VideoCapture(filepath) 

# FaceClssifier 인스턴스 생성
fc = FaceClassifier(threshold, ratio, result_dir)

fingerprints = dict()

frames = []
frame_count = 0
cnt = 0

running = True

total_start = time.time()
while running:
    ret, frame = video.read()

    if frame is None:
        break

    seconds = int(round(frame_count / fps, 3))
    # print(f"Running in {seconds} sec in video...")
    if seconds > stop > 0:
        break
    if seconds < skip:
        continue
    if frame_count % capture_interval == 0:
        # resize
        if frame.shape[0] > 1000:
            frame = cv2.resize(frame, None, fx=0.6, fy=0.6)
        frames.append(frame)
        
    if len(frames) == frame_batch_size:
        # 얼굴 디텍션 -> 얼굴 인코딩(frame, face_boxes) -> 옷 디텍션 -> 옷 인코딩(read_frames 코드 + _frame_worker 코드 + get_model 코드 + fingerprint 코드) -> 이미지 저장
        # 반환: 상체 이미지(파일명): [얼굴 인코딩 | 옷 인코딩] (fingerprint)
        frame_fingerprints = fc.detect_faces(frames,frame_batch_size)
        print(frame_fingerprints)
        if frame_fingerprints:
            fingerprints.update(frame_fingerprints)
            print('Face images: ', len(fingerprints))
            cnt += 1
            print('frame_cnt: ', cnt)
            print()
        
        frames = []
        
    if len(fingerprints) >= 150:
        break
    
    frame_count += 1



# restore SIGINT (^C) handler
running = False
video.release()
print()

# pdb.save_db(result_dir)
# pdb.print_persons()

total_end = time.time()
print('Inference time: ',total_end-total_start)
print("Captured frames : ", frame_count)

image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-0.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-1.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-2.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-3.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-4.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-5.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-6.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.626-7.png
image saved path:  /opt/ml/input/final-project-level3-cv-10/data/sample1_0-300s_test2/20220601_080526.62


KeyboardInterrupt



In [ ]:
filepath = video_paths[file_num]
result_dir = filepath[:-4] + '_test2'

In [ ]:
# 클러스터링(fingerprints) 후 결과 출력
clusters = calc.cluster(fingerprints, sim=0.60, method='single', min_csize=3) # 높일수록 엄격하게

postproc.make_links(clusters, os.path.join(result_dir, 'imagecluster/clusters'))

images = icio.read_images(result_dir, size=(224,224))

fig, ax = postproc.plot_clusters(clusters, images)
fig.savefig(os.path.join(result_dir, 'imagecluster/_cluster.png'))
postproc.plt.show()